In [3]:
import sys
sys.path.append('../')
from collections import defaultdict

from src.experiment_iterator import ExperimentState
from src.task_loaders import TRAIN, TEST, ALL
from src.models.model_loaders import *
from src.utils import *

from data.compositional_graphics.make_tasks import *

# All of the model loaders we import.
from data.compositional_graphics.grammar import *
from data.compositional_graphics.encoder import *

from src.models.laps_dreamcoder_recognition import *
from src.models.seq2seq import *

### Compositional Graphics (LAPS Graphics)

In [4]:
import json 

DEFAULT_CONFIG_DIR = "../experiments/configs"
GRAPHICS_CONFIG = "seq2seq_language_only_compositional_graphics_200_human.json"

def load_config_from_file(config_dir, config_file):
    config_full_path = os.path.join(config_dir, config_file)
    with open(config_full_path) as f:
        return json.load(f)


config = load_config_from_file(config_dir=DEFAULT_CONFIG_DIR, config_file=GRAPHICS_CONFIG)
experiment_state = ExperimentState(config)

In [42]:
from dreamcoder.program import *
nameSimplification = {
        "fix1": 'Y',
        "tower_loopM": "for",
        "tower_embed": "get/set",
        "moveHand": "move",
        "reverseHand": "reverse",
        "logo_DIVA": '/',
        "logo_epsA": 'ε',
        "logo_epsL": 'ε',
        "logo_IFTY": '∞',
        "logo_forLoop": "for",
        "logo_UA": "2π",
        "logo_FWRT": "move",
        "logo_UL": "1",
        "logo_SUBA": "-",
        "logo_ZL": "0",
        "logo_ZA": "0",
        "logo_MULL": "*",
        "logo_MULA": "*",
        "logo_PT": "pen-up",
        "logo_GETSET": "get/set"
    }

def lb(s,T=20):
        s = s.split()
        l = []
        n = 0
        for w in s:
            if n + len(w) > T:
                l.append("<br />")
                n = 0
            n += len(w)
            l.append(w)
        return " ".join(l)
    
name = {}
simplification = {}
depth = {}
def getName(p):
    p = p.uncurry().betaNormalForm()
    p = prettyProgram(p, Lisp=True)
    p = str(p)
    for original, simple in nameSimplification.items():
        p = p.replace(original, simple)
    return p

language_program_data = []
for split in experiment_state.tasks:
    for task in experiment_state.tasks[split]:
        task_data = {}
        task_data["split"] = split
        task_data["synthetic language"] = task.name.split("_")[0]
        task_data["human language"] =  "\n".join(experiment_state.task_language[split][task.name])
        
        task_data["gt_initial_program"] = getName(task.supervision)
        
        language_program_data.append(task_data)
        

In [46]:
from IPython.display import display, HTML
import pandas as pd
df = pd.DataFrame(language_program_data) 

with open("visualize_language_programs_graphics.html", "w") as f:
    f.write(df.to_html())
    
display(HTML(df.to_html().replace("\\n","<br>")))

,split,synthetic language,human language,gt_initial_program
0,train,a small triangle,small trianglesmall thick lined triangle in the center of pagesmall triangle,(λ (x) (for 3 (λ (y z) (move (* 1 1) (/ 2π 3) z)) x))
1,train,a medium triangle,triangletrianglesa medium triangle,(λ (x) (for 3 (λ (y z) (move (* 1 2) (/ 2π 3) z)) x))
2,train,a small 8 gon,a medium octagon1 large 8 gon in center of pagemedium octagon,(λ (x) (for 8 (λ (y z) (move (* 1 1) (/ 2π 8) z)) x))
3,train,a small 9 gon next to a medium square,a large nonagon next to a medium square1 medium 8 gon with 1 flat side up against the left top side of a medium square9 gon touching a square,(λ (x) (for 9 (λ (y z) (move (* 1 1) (/ 2π 9) z)) (pen-up (λ (u) (move (* 1 2) 0 u)) (for 4 (λ (v w) (move (* 1 2) (/ 2π 4) w)) x))))
4,train,4 concentric square s,nested squaresquare of square s4 nested square s,(λ (x) (for 5 (λ (y z) (get/set (λ (u) (for 4 (λ (v w) (move (* 1 y) (/ 2π 4) w)) u)) z)) x))
5,train,4 concentric square s,4 nested small square s4 nested box4 square s inside each other wifi symbol,(λ (x) (for 5 (λ (y z) (get/set (λ (u) (for 4 (λ (v w) (move (* 1 y) (/ 2π 4) w)) u)) z)) x))
6,train,7 sided snowflake with a medium circle and a small 5 gon as arms,ring of 7 overlapped circle s surrounded by 7 5 gon ssmall space in middle with 7 small circle s around it and 7 5 gon s outside circle sa snowflake with 7 pentagon leaves on the outside with 7 small circle s on the inside7 circle s in the middle with 7 pentagon s on the outside,(λ (x) (for 7 (λ (y z) (get/set (λ (u) (pen-up (λ (v) (move 1 0 v)) (for ∞ (λ (w a) (move (* ε 2) ε a)) (for ∞ (λ (b c) (move (* ε 2) ε c)) (pen-up (λ (d) (move 1 0 d)) (for 5 (λ (e f) (move (* 1 1) (/ 2π 5) f)) u)))))) (move 0 (/ 2π 7) z))) x))
7,train,a short line next to a small 6 gon,a small dash and a medium 6 gonshoirt loine with a 5 sided medium gonline on left and 6 gon on the right,(λ (x) (move 1 0 (pen-up (λ (y) (move (* 1 2) 0 y)) (for 6 (λ (z u) (move (* 1 1) (/ 2π 6) u)) x))))
8,train,6 sided snowflake with a short line and a small semicircle as arms,line art6 candy cane pinwheel,(λ (x) (for 6 (λ (y z) (get/set (λ (u) (move (* 1 1) 0 (for ∞ (λ (v w) (move (* ε 1) ε w)) u))) (move 0 (/ 2π 6) z))) x))
9,train,6 small 5 gon s in a row,6 overlapping horizontal 5 gon s6 overlapping pentagon s in a line6 merged pentagon6 pentagon s in a row,(λ (x) (for 6 (λ (y z) (get/set (λ (u) (for 5 (λ (v w) (move (* 1 1) (/ 2π 5) w)) u)) (pen-up (λ (a) (move 1 0 a)) z))) x))
